In [ ]:
import mne
import os
from tqdm import tqdm_notebook
import neurokit2 as nk
import numpy as np
import yasa
from scipy.signal import welch

In [ ]:
import heartpy as hp

In [ ]:
ratings = pd.read_excel('ratings/Result movies.xlsx')

In [ ]:
n_subjects = 21
matrix = pd.DataFrame()
for Subject in tqdm_notebook(range(1, n_subjects + 1)):
    filtered = mne.io.read_raw(
        'raw_filtered_fif/S{}_filtered_raw.fif'.format('0' + str(Subject) if Subject < 10 else Subject))
    raw = mne.io.read_raw('raw_full/' + (('00' + str(Subject) if Subject < 10 else '0' + str(Subject)) + 'a' if Subject ==
                          2 or Subject == 5 else ('00' + str(Subject) if Subject < 10 else '0' + str(Subject))) + '.vhdr')
    annot = filtered.annotations
    n_events = 12 if Subject == 11 else 16
    for i in range(0, n_events, 2):
        film = annot[i]['description'].split('_')[-1]
        try:
            watch = raw.copy().crop(
                annot[i]['onset'], annot[i]['onset'] + annot[i]['duration'])
        except Exception as e:
            watch = raw.copy().crop(
                annot[i]['onset'], annot[i]['onset'] + annot[i]['duration'] - 0.01)

        filtered = hp.hampel_filter(watch['Pulse'][0][0], filtsize=6)
        try:
            wd, m = hp.process(hp.scale_data(filtered), 1000)
        except Exception as e:
            continue

        m['Film'] = film
        m['Subject'] = Subject
        m['Labels'] = ratings.loc[Subject - 1][film + '.mp4']
        matrix = pd.concat([matrix, pd.DataFrame(m, index=[0])], axis=0)